In [ ]:
# Jupyter Notebook에서 출력 생략을 방지
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import import_ipynb
import utility
import Model
import EWC

In [ ]:
from keras import backend as K
from EWC import evaluate
from utility import LoadDataNoDefCW_EWC
from Model import DFNet
import random
from keras.utils import np_utils
from keras.optimizers import Adamax
import numpy as np
import os
import tensorflow as tf


random.seed(0)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Use only CPU
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

description = "Training and evaluating DF model for closed-world scenario on non-defended dataset"

print(description)
# Training the DF model
NB_EPOCH = 30   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 128 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 10000 # Packet sequence length
OPTIMIZER = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) # Optimizer

NB_CLASSES = 95 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

In [ ]:
# Data: shuffled and split between train and test sets
print ("Loading and preparing data for training, and evaluating the model")
X_A_train, X_B_train, y_A_train, y_B_train, X_A_test, X_B_test, y_A_test, y_B_test = LoadDataNoDefCW_EWC()
# Please refer to the dataset format in readme
K.set_image_data_format("channels_last") # tf is tensorflow

In [ ]:
# Building and training model
print ("Building and training DF model")

model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

In [ ]:
# Function to adjust the sequence length to a target length
def adjust_sequence_length(sequence, target_length=10000, padding_value=-1):
    if len(sequence) < target_length:
        # If the sequence is shorter than the target length, pad with the specified value
        sequence = sequence + [padding_value] * (target_length - len(sequence))
    else:
        # If the sequence is longer than the target length, truncate it
        sequence = sequence[:target_length]
    return sequence

# Adjust sequence lengths for each dataset
X_A_train = np.array([adjust_sequence_length(seq) for seq in X_A_train])
X_B_train = np.array([adjust_sequence_length(seq) for seq in X_B_train])
X_A_test = np.array([adjust_sequence_length(seq) for seq in X_A_test])
X_B_test = np.array([adjust_sequence_length(seq) for seq in X_B_test])

# Convert data to float32 type
X_A_train = X_A_train.astype('float32')
X_A_test = X_A_test.astype('float32')
X_B_train = X_B_train.astype('float32')
X_B_test = X_B_test.astype('float32')


In [ ]:
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_A_train = X_A_train[:, :,np.newaxis]
X_A_test = X_A_test[:, :,np.newaxis]
X_B_train = X_B_train[:, :,np.newaxis]
X_B_test = X_B_test[:, :,np.newaxis]

print(X_A_train.shape[0], 'train samples')
print(X_A_test.shape[0], 'test samples')
print(X_B_train.shape[0], 'train samples')
print(X_B_test.shape[0], 'test samples')

In [ ]:
# Convert class vectors to categorical classes matrices
y_A_train = np_utils.to_categorical(y_A_train, NB_CLASSES)
y_A_test = np_utils.to_categorical(y_A_test, NB_CLASSES)
y_B_train = np_utils.to_categorical(y_B_train, NB_CLASSES)
y_B_test = np_utils.to_categorical(y_B_test, NB_CLASSES)

In [ ]:
y_A_train.shape

In [ ]:
# 모델 초기화
model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])

# A 데이터로 학습
model.fit(X_A_train, y_A_train, batch_size=BATCH_SIZE, epochs=100, verbose=VERBOSE)

# A 데이터 평가
print("Accuracy on Task A after training on Task A:")
print(evaluate(model, X_A_test, y_A_test))

# B 데이터로 추가 학습 (같은 모델)
model.fit(X_B_train, y_B_train, batch_size=BATCH_SIZE, epochs=100, verbose=VERBOSE)

# B 데이터 평가
print("Accuracy on Task B after training on Task B:")
print(evaluate(model, X_B_test, y_B_test))

# A 데이터로 재평가
print("Accuracy on Task A after training on Task B:")
print(evaluate(model, X_A_test, y_A_test))
